In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import requests
import yfinance as yf

In [2]:
# def get_all_tickers() -> str:
#     url = 'https://quality.data.gov.tw/dq_download_json.php'
#     num_id = '11549'
#     md5_check = 'bb878d47ffbe7b83bfc1b41d0b24946e'
#     r = requests.get(f'{url}?nid={num_id}&md5_url={md5_check}')
#     return [i["證券代號"] for i in r.json()]
def get_all_tickers() -> str:
    url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY_ALL?response=open_data'
    df_tickers_data = pd.read_csv(f'{url}')
    print(df_tickers_data["證券代號"].values)
    return list(df_tickers_data["證券代號"].values)

In [7]:
tickers = ['2330.TW', '0050.TW']#[ i+'.TW' for i in get_all_tickers() ]
stock = yf.download(
    tickers=tickers,
    start="2022-12-05",
    end="2022-12-06",
    interval="1h", # minute
    ignore_tz=False
)
stock

[*********************100%***********************]  2 of 2 completed


Adj Close               Close                High  \
                              0050.TW 2330.TW     0050.TW 2330.TW     0050.TW   
2022-12-05 09:00:00+08:00  118.150002   495.0  118.150002   495.0  118.500000   
2022-12-05 10:00:00+08:00  117.849998   494.5  117.849998   494.5  118.349998   
2022-12-05 11:00:00+08:00  117.800003   493.0  117.800003   493.0  118.099998   
2022-12-05 12:00:00+08:00  117.300003   492.0  117.300003   492.0  117.849998   
2022-12-05 13:00:00+08:00  117.500000   491.0  117.500000   491.0  117.500000   

                                          Low                Open          \
                          2330.TW     0050.TW 2330.TW     0050.TW 2330.TW   
2022-12-05 09:00:00+08:00   497.5  117.199997   491.0  117.300003   491.5   
2022-12-05 10:00:00+08:00   497.0  117.750000   494.0  118.099998   495.0   
2022-12-05 11:00:00+08:00   495.0  117.750000   493.0  117.849998   495.0   
2022-12-05 12:00:00+08:00   494.0  117.250000   491.0  117.800003   493.0   
2022-12-05 13:00:00+08:00   492.5  117.300003   491.0  117.300003   492.5   

                            Volume           
                           0050.TW  2330.TW  
2022-12-05 09:00:00+08:00        0        0  
2022-12-05 10:00:00+08:00  2472159  4464815  
2022-12-05 11:00:00+08:00  1733777  3112476  
2022-12-05 12:00:00+08:00  2597536  4911480  
2022-12-05 13:00:00+08:00   782648  2411133

In [12]:
date.today().strftime("%Y-%m-%d")

'2022-12-06'

In [16]:
from datetime import date, timedelta
print('download_yesterday')
# try:
#     tickers = get_all_tickers()
# except:
#     print('quality.data.gov.tw cannot get results')
#     return
tickers_TW = ['2330.TW', '0050.TW']#[ i+'.TW' for i in tickers ]
yesterday = date.today() - timedelta(days=1)
stock = yf.download(
    tickers=tickers_TW,
    start=yesterday.strftime("%Y-%m-%d"),
    end=date.today().strftime("%Y-%m-%d"),
    interval="1h", # minute
    ignore_tz=False
)
df_stocks = process_dataframe(stock, tickers)
stocks_dicts = df_stocks.to_dict(orient='records')
stocks_dicts

download_yesterday
[*********************100%***********************]  2 of 2 completed


[{'datetime': Timestamp('2022-12-05 09:00:00+0800', tz='Asia/Taipei'),
  'ticker': '2330.TW',
  'adjclose': 495.0,
  'close': 495.0,
  'high': 497.5,
  'low': 491.0,
  'open': 491.5,
  'volume': 0},
 {'datetime': Timestamp('2022-12-05 10:00:00+0800', tz='Asia/Taipei'),
  'ticker': '2330.TW',
  'adjclose': 494.5,
  'close': 494.5,
  'high': 497.0,
  'low': 494.0,
  'open': 495.0,
  'volume': 4464815},
 {'datetime': Timestamp('2022-12-05 11:00:00+0800', tz='Asia/Taipei'),
  'ticker': '2330.TW',
  'adjclose': 493.0,
  'close': 493.0,
  'high': 495.0,
  'low': 493.0,
  'open': 495.0,
  'volume': 3112476},
 {'datetime': Timestamp('2022-12-05 12:00:00+0800', tz='Asia/Taipei'),
  'ticker': '2330.TW',
  'adjclose': 492.0,
  'close': 492.0,
  'high': 494.0,
  'low': 491.0,
  'open': 493.0,
  'volume': 4911480},
 {'datetime': Timestamp('2022-12-05 13:00:00+0800', tz='Asia/Taipei'),
  'ticker': '2330.TW',
  'adjclose': 491.0,
  'close': 491.0,
  'high': 492.5,
  'low': 491.0,
  'open': 492.5,
  '

In [15]:
def process_dataframe(stock, tickers):
    columns = [ 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume' ]
    stocks = []
    for ticker in tickers:
        tmp_stock = stock[ [ (column, ticker) for column in columns] ].copy()
        tmp_stock.columns = columns
        tmp_stock['Ticker'] = ticker
        tmp_stock = tmp_stock[ ["Ticker"]+columns ]
        tmp_stock.reset_index(names='datetime', inplace=True)
        tmp_stock.dropna(subset=columns, how="all", axis=0, inplace=True)
        stocks += [ tmp_stock ]

    stock = pd.concat(stocks)
    stock.columns = [ ''.join(x.lower().split()) for x in stock.columns.tolist() ]
    return stock
# stock = process_dataframe(stock)
# stock.head()

In [17]:
from sqlalchemy.dialects.postgresql import insert


def _postgres_do_nothing_on_conflict(table, conn, keys, data_iter):
    print(type(table.table),table.table.name)
    print(table.table)
    data = [dict(zip(keys, row)) for row in data_iter]
    # insert_statement = insert(table.table).values(data)
    # upsert_statement = insert_statement.on_conflict_do_nothing(
    #     constraint=f"{table.table.name}_pkey"
    # )
    # conn.execute(upsert_statement)

dsn = "postgresql+psycopg2://admin:0000@db:5432/yahoo_stock"
engine = create_engine(dsn)

stock.to_sql(
    "yfinance",
    engine,
    if_exists="append",
    index=False,
    method=_postgres_do_nothing_on_conflict
)

<class 'sqlalchemy.sql.schema.Table'> yfinance
yfinance


In [ ]:
pd.read_sql('select * from yfinance', engine)

In [ ]:
import os 
def create_project_init_structure(root_folder:str):
    __forder_list = ['configs', 'dao', 'init', 'utils', 'jobs']
    for dir_name in __forder_list:
        directory = os.path.join(
            root_folder, 
            dir_name)
        if not os.path.exists(directory):
            os.makedirs(directory)
            init_file_path = os.path.join(directory,'__init__.py')
            with open(init_file_path, 'w') as f:
                f.write("#!/usr/bin/python3")
        else:
            print(f'Already exists: {directory}')
create_project_init_structure("../")